In [ ]:
import os, sys, glob, re, math, pickle
import scprep, magic, phate
import numpy as np
import pandas as pd
from scipy import sparse as sp
import time,random,datetime
import scanpy as sc
import anndata
from typing import Dict, Optional
import tables
import seaborn as sns
from bbknn import bbknn
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext memory_profiler

# reproducibility
rs = np.random.seed(42)

# fps
dfp = '/gpfs/gibbs/pi/lim_janghoo/cl2292/'
pfp = '/gpfs/gibbs/pi/lim_janghoo/cl2292/SCA1_OL/results/'
pdfp = '/gpfs/gibbs/pi/lim_janghoo/cl2292/SCA1_OL/data/'
sc.settings.figdir = pfp

# settings
plt.rc('font', size = 8)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['text.usetex']=False
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
sc.set_figure_params(dpi=300,dpi_save=600,
                     frameon=False,
                     fontsize=8)
plt.rcParams['savefig.dpi']=600
sc.settings.verbosity=2
sc._settings.ScanpyConfig.n_jobs=-1

In [ ]:
if True :
    start = time.time()
    backed=None # None if not
    fname='250414_OL-SCA1-cKI_imp.h5ad' # for full, can maybe get away with ~300G
    %memit adata = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))

In [ ]:
adata = adata[(adata.obs['ctype']=='OPC')|(adata.obs['ctype']=='OL'),:]

In [ ]:
adata.X=adata.layers["counts"]
sc.pp.normalize_total(adata)
sc.pp.sqrt(adata,chunked=True,chunk_size=10000)
adata.raw = adata


# calc embeddings for batch corrected
start = time.time()
print('starting embeddings...')
sc.tl.pca(adata,n_comps=100)
#sc.external.pp.bbknn(adata,batch_key='batch')
bbknn(adata,batch_key='batch') #pip install bbknn
sc.tl.leiden(adata,resolution=1) 
sc.tl.umap(adata)
sc.pl.umap(adata, color = ['genotype','leiden'])

In [ ]:
markers = {
    'OPC':['Pdgfra','Cspg4'],
    'OPC1':['Pcdh15'], #,'Ptprz1'
    'OPC2':['Caln1'], #'Car10'
    'COP':['Fyn','Tcf7l2'], #'Bmp4','Neu4'
    'NFOL':['Man1a','Synpr'], #,'Prom1','Tspan2' #marques 2018, Matson 2022; Hilscher2022 'Enpp6','Mpzl1','Itpr2'  
    'OL':['Plp1','Mag','Mog','Mbp','Mobp','Opalin'], #,'Hapln2','Dpy19l1'
    'MOL1':['Egr2'], #'Kirrel3','Egr2',
    'MOL2': ['Klk6',], #'Cyp27a1','Rab37'
    'MOL4': ['Ttr',],
    'MOL5/6' : ['Ptgds'], #,'Grm3','Car2'
    'OL3':['Fgf14','Rbfox1','Nrxn1']}

#sc.tl.dendrogram(adata, groupby = 'ctype')
sc.pl.dotplot(adata, markers, groupby = 'leiden', standard_scale='var', use_raw=False, dendrogram=True,
#              save = '_241025_KI_OLsubtype.pdf'
             )

In [ ]:
# annotate cell type
opc1 = ['5']
opc2 = ['6']

cop = ['8']
nfol = ['9']

ol1 = ['1','2']
ol2 = ['0']
ol3 = ['3','4','7']

ctype = []
for i in adata.obs['leiden'] : # verbose loop for quality-assurance
    if i in opc1 :
        ctype.append('OPC1') 
    elif i in opc2 :
        ctype.append('OPC2')
    elif i in cop :
        ctype.append('COP')
    elif i in nfol :
        ctype.append('NFOL')
    elif i in ol1 :
        ctype.append('OL1')
    elif i in ol2 :
        ctype.append('OL2')
    elif i in ol3 :
        ctype.append('OL3')
    else:
        raise ValueError('Encountered unclassifiable cell type for sample {}'.format(i))
adata.obs['ctype']=ctype
sc.pl.umap(adata, color = ['ctype'])

cmap_ctype={'OPC1': '#e41a1c',
              'OPC2':'#377eb8',
              'COP':'#4daf4a',
              'NFOL':'#f781bf',
              'OL1':'#984ea3',
              'OL2':'#ff7f00',
              'OL3':'#a65628'}

adata.uns['ctype_colors']=np.array(list(cmap_ctype.values()),dtype=object)
adata.obs['ctype']=adata.obs['ctype'].cat.reorder_categories(new_categories=('OPC1','OPC2','COP','NFOL','OL1','OL2','OL3'),
                                         ordered=True)
sc.pl.umap(adata, color = ['ctype'])

cmap_genotype={'SCA1-fl/+': '#4683B5',
            'SCA1-fl/NG2-Cre':'#FFA600'}

adata.uns['genotype_colors']=np.array(list(cmap_genotype.values()),dtype=object)
adata.obs['genotype']=adata.obs['genotype'].cat.reorder_categories(new_categories=list(cmap_genotype.keys()),
                                         ordered=True)

In [ ]:
random_indices = np.random.permutation(list(range(adata.shape[0])))
size = 40

sc.pl.umap(adata[random_indices, :], color = 'genotype', size = size,
           save = '_250414_OL-SCA-cKI_OLsub_genotype.pdf'
          )
sc.pl.umap(adata[random_indices, :], color = 'ctype', size = size,
           save = '_240414_OL-SCA-cKI_OLsub_ctype.pdf'
          )

In [ ]:
cell_counts = adata.obs.groupby(['ctype','genotype','batch']).size().reset_index(name='cell_count')
total_counts = adata.obs.groupby(['genotype','batch']).size().reset_index(name='total_cells')
cell_counts = cell_counts.merge(total_counts, on=['genotype','batch'])
cell_counts['proportion(%)'] = cell_counts['cell_count']*100 / cell_counts['total_cells']
cell_counts = cell_counts.sort_values(by=['ctype', 'genotype','batch'])
cell_counts
cell_counts.to_csv(os.path.join(pfp,'250415_OL-SCA1-cKI_OL sub proportion.csv'),index=False)

In [ ]:
adata.write(os.path.join(pdfp,'250415_OL-SCA1-cKI_OLsub.h5ad'))

# Imputation and DGE analysis

In [ ]:
# Import packages
import os, sys, glob, re, math, pickle
import phate, scprep, magic
import graphtools as gt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time,random,datetime
import networkx as nx
from sklearn import metrics
from sklearn import model_selection
from scipy import sparse
from scipy.stats import mannwhitneyu, tiecorrect, rankdata
from statsmodels.stats.multitest import multipletests
import scanpy as sc
from sklearn.dummy import DummyClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import SpectralClustering, OPTICS, cluster_optics_dbscan, AgglomerativeClustering
from bbknn import bbknn
import warnings
%matplotlib inline
%load_ext memory_profiler
import logging

# settings
plt.rc('font', size = 8)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['text.usetex']=False
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
sc.set_figure_params(dpi=300,dpi_save=600,
                     frameon=False,
                     fontsize=8)
plt.rcParams['savefig.dpi']=600
sc.settings.verbosity=2
sc._settings.ScanpyConfig.n_jobs=-1

# reproducibility
rs = np.random.seed(42)

# utils
def mwu(X,Y,gene_names,correction=None,debug=False) :
    '''
    Benjamini-Hochberg correction implemented. Can change to Bonferonni
    gene_names (list)
    if X,Y single gene expression array, input x.reshape(-1,1), y.reshape(-1,1)
    NOTE: get zeros sometimes because difference (p-value is so small)
    '''
    p=pd.DataFrame()
    print('Mann-Whitney U w/Benjamini/Hochberg correction\n')
    start = time.time()
    for i,g in enumerate(gene_names) :
        if i==np.round(np.quantile(np.arange(len(gene_names)),0.25)) :
            print('... 25% completed in {:.2f}-s'.format(time.time()-start))
        elif i==np.round(np.quantile(np.arange(len(gene_names)),0.5)) :
            print('... 50% completed in {:.2f}-s'.format(time.time()-start))
        elif i==np.round(np.quantile(np.arange(len(gene_names)),0.75)) :
            print('... 75% completed in {:.2f}-s'.format(time.time()-start))
        p.loc[i,'Gene']=g
        if (tiecorrect(rankdata(np.concatenate((np.asarray(X[:,i]),np.asarray(Y[:,i])))))==0) :
            if debug :
                print('P-value not calculable for {}'.format(g))
            p.loc[i,'pval']=np.nan
        else :
            _,p.loc[i,'pval']=mannwhitneyu(X[:,i],Y[:,i]) # continuity correction is True
    print('... mwu computed in {:.2f}-s\n'.format(time.time() - start))
    # ignore NaNs, since can't do a comparison on these (change numbers for correction)
    p_corrected = p.loc[p['pval'].notna(),:]
    new_pvals = multipletests(p_corrected['pval'],method='fdr_bh')
    p_corrected['pval_corrected'] = new_pvals[1]
    return p_corrected

def log2aveFC(X,Y,gene_names,AnnData=None) :
    '''not sensitivity to directionality due to subtraction
    X and Y full arrays, subsetting performed here
    `gene_names` (list): reduced list of genes to calc
    `adata` (sc.AnnData): to calculate reduced list. NOTE: assumes X,Y drawn from adata.var_names
    '''
    if not AnnData is None :
        g_idx = [i for i,g in enumerate(AnnData.var_names) if g in gene_names]
        fc=pd.DataFrame({'Gene':AnnData.var_names[g_idx],
                         'log2FC':np.log2(X[:,g_idx].mean(axis=0)) - np.log2(Y[:,g_idx].mean(axis=0))}) # returns NaN if negative value 
    else :
        fc=pd.DataFrame({'Gene':gene_names,
                         'log2FC':np.log2(X.mean(axis=0)) - np.log2(Y.mean(axis=0))})
    return fc


# fps
dfp = '/vast/palmer/pi/lim_janghoo/cl2292/'
pfp = '/vast/palmer/pi/lim_janghoo/cl2292/SCA1_OL/results/'
pdfp = '/vast/palmer/pi/lim_janghoo/cl2292/SCA1_OL/data/'
sc.settings.figdir = pfp

In [ ]:
#Imputation by genotype & timepoint

wt30 = adata[(adata.obs['genotype']=='SCA1-fl/+'), :]
mut30 = adata[(adata.obs['genotype']=='SCA1-fl/NG2-Cre'), :]

#k=45, t=3
print('Starting imputation for {}\n'.format('30wk SCA1-fl/+'))
tic = time.time()


wt30.obs['value'] = 0
sc.pp.pca(wt30)
sc.pp.neighbors(wt30, n_pcs=45)


# MAGIC
G = gt.Graph(data=wt30.obsp['connectivities']+sparse.diags([1]*wt30.shape[0],format='csr'),
             precomputed='adjacency',
             use_pygsp=True)
G.knn_max = None

magic_op=magic.MAGIC().fit(X=wt30.X,graph=G) # running fit_transform produces wrong shape
wt30.layers['imputed']=magic_op.transform(wt30.X,genes='all_genes')

print('\n  imputation in {:.2f}-min'.format((time.time() - tic)/60))


print('\n Starting imputation for {}\n'.format('30wk SCA1-fl/NG2-Cre'))
tic = time.time()

mut30.obs['value'] = 0
sc.pp.pca(mut30)
sc.pp.neighbors(mut30, n_pcs=45)

# MAGIC
G = gt.Graph(data=mut30.obsp['connectivities']+sparse.diags([1]*mut30.shape[0],format='csr'),
             precomputed='adjacency',
             use_pygsp=True)
G.knn_max = None

magic_op=magic.MAGIC().fit(X=mut30.X,graph=G) # running fit_transform produces wrong shape
mut30.layers['imputed']=magic_op.transform(mut30.X,genes='all_genes')

print('\n  imputation in {:.2f}-min'.format((time.time() - tic)/60))

adata =wt30.concatenate([mut30] ,batch_key='concat')

# save data objects
adata.write(os.path.join(pdfp,'250415_OL-SCA1-cKI_OLsub_imp.h5ad'))
print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

In [ ]:
## EMD; IMP
wt = adata[(adata.obs['genotype']=='SCA1-fl/+'), :]
mut = adata[(adata.obs['genotype']=='SCA1-fl/NG2-Cre'), :]

if True :
    dge_grandtotal = time.time()
    group='ctype'
    fname = 'OL-SCA1-cKI OL subtype' 
    dge = pd.DataFrame()
    for t in ['30wk'] :
        print('Evaluating {}'.format(t))
        t_total = time.time()
        dge_total = time.time()
        start_t=time.time()
        
        # up down dichotomy
        print('\n--------')
        print('...')
        print('--------\n')
#        dge = pd.DataFrame()
        for i in wt.obs[group].unique():
            start = time.time()
            print('\n{}, SCA1 vs WT'.format(i))
            print('----')
            X = wt[(wt.obs[group]==i)&(wt.obs['timepoint']==t), :].layers['imputed']
            Y = mut[(mut.obs[group]==i)&(mut.obs['timepoint']==t), :].layers['imputed']
            

            X = np.asarray(X)
            Y = np.asarray(Y)
        
            print('    Ncells in X:{}'.format(X.shape[0]))
            print('    Ncells in Y:{}\n'.format(Y.shape[0]))            
            
            emd = scprep.stats.differential_expression(Y,X,
                                                       measure = 'emd',
                                                       direction='both',
                                                       gene_names=wt.var_names,
                                                       n_jobs=-1)
            
            # mann-whitney u, corrected p-values
            p = mwu(Y,X, wt.var_names)
            emd['Gene']=emd.index
            emd=emd.drop(columns='rank')
            fc = log2aveFC(Y,X,wt.var_names.to_list())
            gene_mismatch = fc['Gene'].isin(p['Gene'])
            if gene_mismatch.any():
                fc = fc.loc[gene_mismatch,:]
                warnings.warn('Warning: {} genes dropped due to p-val NA.'.format((gene_mismatch==False).sum()))
            dt = pd.merge(p,fc,how='left',on="Gene")
            gene_mismatch = emd['Gene'].isin(p['Gene'])
            if gene_mismatch.any():
                emd = emd.loc[gene_mismatch,:]
            dt = pd.merge(dt,emd,how='left',on='Gene')
            dt['Cell type']=[i]*dt.shape[0]
            dt['timepoint']=[str(t)]*dt.shape[0]
            dt['nlog10pvalcorrected']=(-1)*np.log10(dt['pval_corrected'])
            dge = pd.concat([dge,dt], ignore_index=True)
            print('... computed in {:.2f}-s'.format(time.time()-start))
        print('\nFinished timepoint {} in {:.2f}-min'.format(t,(time.time()-start_t)/60))  
    dge.to_csv(os.path.join(pfp,'250415_dge_'+fname+'.csv'),index=False)
#     dgeup = dge.loc[dge['emd']>0,:] # take only 'up' (switch for down)
#     dgedown = dge.loc[dge['emd']<0,:] # take only 'down'
#     dgeup.to_csv(os.path.join(pfp,'240723_dge_'+fname+'_SCA1flwCre_up.csv'),index=False)
#     dgedown.to_csv(os.path.join(pfp,'240723_dge_'+fname+'_SCA1flwCre_down.csv'),index=False)
        

    print('DGE finished in {:.2f}-min'.format((time.time()-dge_grandtotal)/60))
